## Objetivo: Generar los ficheros CSV de Train , Test y Val
Agrupa los paths completos de un directorio de imágenes agrupadas en clases (directorios) en tres ficheros CSV denominados train, tes y val. El numero de elementos para las clases de Test y Val, se pasan por parámetros; el resto se utiliza para la clase Train.

### Condiciones:
#### DIRECTORIO SOURCE: Comprobación fichero (test_RawData_Dir.ipynb)
1. El directorio con los datos RAW solo ha de contener un subdirectorio por clase dentro de los cuales puede haber cualquier estructura y contenido aunque sólo se utilizarán los ficheros de imágenes con extensión  EXT
3. Los nombres de los directorios/subdirectorios no pueden contener espacios en blanco.

#### DIRECTORIO SINK: Comprobación función (test_sinkDir)
1. El directorio destino debe estar vacío, en otro caso se eliminarán los elementos que haya en él

### Pruebas:
1. Antes de comenzar con la creación de los ficheros CSV se comprobará que el directorio destino está vacío 

### Salida
El programa ha de dividir las imágenes de cada clase en tres subconjuntos no solapados en función de unas proporciones que se establecerán al inicio del programa. 



### CONSTANTES / PARAMETROS  DE EJECUCION

In [1]:
## Path al directorio fuente (donde están los directorios de las clases con las imágenes)
SOURCEDIR = '/home/manuel/Escritorio/limpieza dataset v2/limpiado/en_el_servidor'

## Path al directorio destino (donde están los directorios de Train, Test y Val)
SINKDIR   = '/home/manuel/Documentos/servidor/resnet50v2_vacias/dataset/'


REPLACE_DIR = '/media/discoA/datasetBiodiversidad'

## Numero de imágenes por clase en Test y Val
NUM_TEST_EXAMPLES_PER_CLASS = 0
NUM_VAL_EXAMPLES_PER_CLASS  = 200
NUM_MAX_EXAMPLES_TRAIN = 20000

#################################################################
#  Parámetros para los test de comprobación del conjunto de datos

## Extensión de las imágenes
EXT = ['JPEG', 'JPG', 'PNG', 'png', 'jpeg', 'jpg']

### PAQUETES A CARGAR

In [2]:
import glob

import cv2
import matplotlib.pyplot as plt

%matplotlib inline

import numpy as np

import os
import pandas as pd
import random 

#import time

import shutil

### FUNCIONES AUXILIARES

In [3]:
#################################################################################
## get_class_dirs
# Obtinene los directorios (sin los paths) que hay en el directorio directory
def get_class_dirs(directory):
    dirs = [os.path.basename(x[0:-1]) for x in glob.glob(directory + os.sep + '*' + os.sep)]        
    return(dirs)




#################################################################################
## get_recursive_list_of_files
#     For the given path, get the List of all files in the directory tree 
def get_recursive_list_of_files(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + get_recursive_list_of_files(fullPath)
        else:
            allFiles.append(fullPath.replace(SOURCEDIR, REPLACE_DIR))
                
    return allFiles


#################################################################################
## get_files_by_EXT
#   for a list of files return only the files with extension EXT 
def get_files_by_EXT(listofPaths, extension):
    allFiles = [x for x in listofPaths if x.split('.')[-1] in extension]            
    return allFiles


#################################################################################
## limpia_sinkDir
#  Borra recursivamente todo el contenido del directorio sinkDir. 
#  Básicamente borra y vuelve a crear el directorio

def limpia_sinkDir(strFolder):
    shutil.rmtree(strFolder + os.sep)
    os.mkdir(strFolder)
    listDir = glob.glob(strFolder + os.sep + '*')
    return(len(listDir))



#################################################################################
## divide_list_FromProb
# Dada una lista devuelve tres listas aleatorias en función de las proporciones
# pasadas por parámetros

def divide_list_FromProb(lista, ratios):
    lista1 = []
    lista2 = []
    lista3 = []
    ratios = np.asarray(ratios)
    # comprobamos que pasamos tres elementos en probs 
    assert len(ratios) == 3,"ERROR(divide_list_FromProb): param2(Ratios) must have 3 elements"
    # comprobamos que son positivos  
    assert np.sum(ratios) == np.sum(np.abs(ratios)),"ERROR(divide_list_FromProb): Ratio lower than 0"
    
    lista = np.asarray(lista)
    if len(lista) != 0:
        # ponemos las proporciones como probabilidades
        ratios  = np.divide(ratios,np.sum(ratios))
        ratios  = np.round(np.cumsum(ratios)*len(lista)).astype(int)
        indices = np.random.permutation(len(lista)).astype(int)
        lista = lista[indices]        
        lista1 = lista[0:ratios[0]]
        lista2 = lista[ratios[0]:ratios[1]]
        lista3 = lista[ratios[1]:ratios[2]]
    return lista1, lista2, lista3



#################################################################################
## divide_list_FromNumExamples
# Dada una lista devuelve tres listas aleatorias disjuntas en la que las dos primeras 
# tienen el numero de  elementos pasados por parámetros y la tercera el resto.

def divide_list_FromNumExamples(lista, numTestExamples, numValExamples):
    ratios = [numTestExamples, numValExamples, len(lista) - numTestExamples - numValExamples]
    lista1, lista2, lista3 = divide_list_FromProb(lista, ratios)
    return lista1, lista2, lista3


### FUNCIONES PRINCIPALES

Comenzamos con la separación del conjunto original en tres subconjuntos Tain, Test y Val.

In [4]:
              
#################################################################################
##  source2sink
#  Copia los elementos de source en los ficheros CSV de train, test y val en sink, 
#  manteniendo repitiendo los directorios de source pero respetando la proporcion de 
#  elementos en train, test y val dictada por ratios 
#

def source2CSVsink(sourceFolder, sinkFolder, numTestExamples, numValExamples):
    
    ERROR = limpia_sinkDir(sinkFolder)
    assert ERROR == 0, 'El directorio sinkFolder no está vacío'
    print('\n')
    
    trainCSV_paths  = []
    trainCSV_labels = []
    valCSV_paths    = []
    valCSV_labels   = []
    testCSV_paths   = []
    testCSV_labels  = []

    # obtenemos una lista con los directorios en sourceFolder (LAS CLASES)
    listaDirs = get_class_dirs(sourceFolder)
    
    # Para cada clase obtenemos los nombres de las imágenes, los dividimos según ratios
    # y copiamos los elementos que corresponden en los directorios específicos de
    # train, test y val
    for folder in listaDirs:
        files1 = get_recursive_list_of_files(sourceFolder + os.sep + folder)
        elementos = get_files_by_EXT(files1, EXT) 
          
        listaTestPath , listaValPath , listaTrainPath = \
            divide_list_FromNumExamples(elementos, numTestExamples, numValExamples)

        listaTrainPath = listaTrainPath[:min(NUM_MAX_EXAMPLES_TRAIN, len(listaTrainPath))]
        
        #        for elemento in listaTrain:
#            fileSource = sourceFolder + os.sep + folder + os.sep + elemento
        listaTestLabel = [folder] * len(listaTestPath)
        listaValLabel = [folder] * len(listaValPath)
        listaTrainLabel = [folder] * len(listaTrainPath)

        assert len(listaTestPath)  == len(listaTestLabel), \
            'different number of elements in listaTestPath and listaTestLabel'
        assert len(listaValPath)   == len(listaValLabel), \
            'different number of elements in listaValPath and listaValLabel'
        assert len(listaTrainPath) == len(listaTrainLabel), \
            'different number of elements in listaTrainPath and listaTrainLabel'

        testCSV_paths   = np.append(testCSV_paths,  listaTestPath)
        testCSV_labels  = np.append(testCSV_labels, listaTestLabel)
        
        valCSV_paths    = np.append(valCSV_paths,  listaValPath)
        valCSV_labels   = np.append(valCSV_labels, listaValLabel)
        
        trainCSV_paths  = np.append(trainCSV_paths,  listaTrainPath)
        trainCSV_labels = np.append(trainCSV_labels, listaTrainLabel)
        

    print('Test(' + str(len(testCSV_paths)) + 
            ')\t\t Val(' + str(len(valCSV_paths)) + 
            ')\t\t Train(' + str(len(trainCSV_paths)) + ')'  ) 
            
            
            
    fileStr =  sinkFolder + os.sep + "test.csv"
    dataTest = {'path':testCSV_paths, 'label':testCSV_labels }
    df = pd.DataFrame(dataTest, columns = ['path','label'])
    df.to_csv(fileStr, index=False)

    fileStr =  sinkFolder + os.sep + "val.csv"
    dataVal = {'path':valCSV_paths, 'label':valCSV_labels }
    df = pd.DataFrame(dataVal, columns = ['path','label'])
    df.to_csv(fileStr, index=False)

    fileStr =  sinkFolder + os.sep + "train.csv"
    dataTrain = {'path':trainCSV_paths, 'label':trainCSV_labels }
    df = pd.DataFrame(dataTrain, columns = ['path','label'])
    df.to_csv(fileStr, index=False)



### EJECUCION




In [5]:
#  Copia los paths de los elementos de SOURCEDIR junto con su clase en los ficheros 
#  CSV de train, test y val en SINKDIR respetando la proporcion de 
#  elementos en train, test y val dictada por RATIOS 
#


source2CSVsink(SOURCEDIR, SINKDIR, NUM_TEST_EXAMPLES_PER_CLASS, NUM_VAL_EXAMPLES_PER_CLASS)



Test(0)		 Val(400)		 Train(40000)
